In [1]:
import findspark
findspark.init('/home/srabin/spark-3.5.1-bin-hadoop3')
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('rf_exercise').getOrCreate()

24/08/07 17:45:05 WARN Utils: Your hostname, DESKTOP-P0KTK6U resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
24/08/07 17:45:05 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/08/07 17:45:06 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/08/07 17:45:06 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/08/07 17:45:06 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
24/08/07 17:45:06 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
24/08/07 17:45:06 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.
24/08/07 17:45:06 WARN Utils: Service 'SparkUI' could not bind on port 4044. Attempting port

In [2]:
data = spark.read.csv('dog_food.csv', inferSchema= True, header = True)

In [3]:
data.printSchema()

root
 |-- A: integer (nullable = true)
 |-- B: integer (nullable = true)
 |-- C: double (nullable = true)
 |-- D: integer (nullable = true)
 |-- Spoiled: double (nullable = true)



In [4]:
data.head(1)

[Row(A=4, B=2, C=12.0, D=3, Spoiled=1.0)]

In [5]:
from pyspark.ml.feature import VectorAssembler

In [6]:
data.columns

['A', 'B', 'C', 'D', 'Spoiled']

In [7]:
assembler = VectorAssembler(inputCols = ['A', 'B', 'C', 'D'], outputCol = 'features')

In [8]:
output = assembler.transform (data)

In [9]:
from pyspark.ml.classification import RandomForestClassifier

In [10]:
rfc = RandomForestClassifier(labelCol = 'Spoiled', featuresCol = 'features')

In [11]:
output.printSchema()

root
 |-- A: integer (nullable = true)
 |-- B: integer (nullable = true)
 |-- C: double (nullable = true)
 |-- D: integer (nullable = true)
 |-- Spoiled: double (nullable = true)
 |-- features: vector (nullable = true)



In [12]:
final_data = output.select('features', 'Spoiled')

In [13]:
final_data.show()

+-------------------+-------+
|           features|Spoiled|
+-------------------+-------+
| [4.0,2.0,12.0,3.0]|    1.0|
| [5.0,6.0,12.0,7.0]|    1.0|
| [6.0,2.0,13.0,6.0]|    1.0|
| [4.0,2.0,12.0,1.0]|    1.0|
| [4.0,2.0,12.0,3.0]|    1.0|
|[10.0,3.0,13.0,9.0]|    1.0|
| [8.0,5.0,14.0,5.0]|    1.0|
| [5.0,8.0,12.0,8.0]|    1.0|
| [6.0,5.0,12.0,9.0]|    1.0|
| [3.0,3.0,12.0,1.0]|    1.0|
| [9.0,8.0,11.0,3.0]|    1.0|
|[1.0,10.0,12.0,3.0]|    1.0|
|[1.0,5.0,13.0,10.0]|    1.0|
|[2.0,10.0,12.0,6.0]|    1.0|
|[1.0,10.0,11.0,4.0]|    1.0|
| [5.0,3.0,12.0,2.0]|    1.0|
| [4.0,9.0,11.0,8.0]|    1.0|
| [5.0,1.0,11.0,1.0]|    1.0|
|[4.0,9.0,12.0,10.0]|    1.0|
| [5.0,8.0,10.0,9.0]|    1.0|
+-------------------+-------+
only showing top 20 rows



In [14]:
rfc_model = rfc.fit(final_data)

In [15]:
final_data.head(1)

[Row(features=DenseVector([4.0, 2.0, 12.0, 3.0]), Spoiled=1.0)]

In [16]:
rfc_model.featureImportances

SparseVector(4, {0: 0.0165, 1: 0.0199, 2: 0.9375, 3: 0.0261})

## this shows the importances of each of A, B, C, and D variables
## SparseVector(4, {0-->A: 0.0165, 1-->B: 0.0199, 2-->C: 0.9375, 3-->D: 0.0261})
### so chemical C is the most important variable